In [1]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch import nn
from PIL import Image
from matplotlib import cm

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=3)

## Load in the relevant data 

In [4]:
base_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Nuclei_Classification/data/"

In [5]:
metadata = pd.read_csv(str(base_dir + "metadata.csv"))

In [6]:
metadata

,Unnamed: 0,paths,sample,labels,heights,widths,dimensions
0,0,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19166.5527, 18793.3867, 19183.4453, 18809.127)"
1,1,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19166.9355, 18781.1016, 19191.5059, 18797.2266)"
2,2,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19299.7715, 18792.2363, 19320.1172, 18811.4316)"
3,3,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19315.127, 18810.6641, 19331.6348, 18830.2422)"
4,4,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19335.0898, 18796.0742, 19348.5273, 18812.584)"
...,...,...,...,...,...,...,...
3746,3746,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,0,32,32,"(11497.5039, 18522.9902, 11504.8701, 18536.8008)"
3747,3747,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,1,32,32,"(11350.1709, 18447.0195, 11373.6514, 18464.9766)"
3748,3748,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,1,32,32,"(11365.8242, 18429.0645, 11388.8457, 18456.2285)"
3749,3749,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,0,32,32,"(11081.5391, 18514.9668, 11092.8975, 18526.7793)"


In [80]:
#transform the function according to the pytorch docs
from torchvision import transforms
#add some image transforms 
# img_size = 224

augmentations = transforms.RandomApply(torch.nn.ModuleList(
            [transforms.RandomRotation((0,315)),
            transforms.ColorJitter(brightness=.3, contrast=.3),
            transforms.RandomSolarize(.3),
            transforms.RandomInvert(), 
            transforms.RandomAdjustSharpness(2),
            ]), p=0.2)

preprocess_augmentation = transforms.Compose([
    #these are the random transforms I got from my other derm project
    augmentations, 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
#it doesn't make sense to do this because the val/test sets also use preprocess. So we need a unique one for train. 
preprocess_normal = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [100]:
#write a different data loader class 
class Patch_Class():
    def __init__(self, metadata, transform=None):
        self.transform = transform
        self.metadata = metadata 
        self.images = {}
        self.load_images()
        
    def load_images(self): #keep all the images in dic
        for path in tqdm(list(self.metadata["paths"])):
            self.images[path] = np.load(path)
        
    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, index):
       
        y_label = self.metadata.iloc[index, 3]
        path = self.metadata.iloc[index, 1]
        #get the image as a numpy array 
        img = self.images[path]
        
#         turn the array into a PIL image, so that it can be resized and transformed
        img = Image.fromarray(img.astype('uint8'), 'RGB') #this here takes a lot of time, and it considerably slows training
        
        #get y_label and one hot encode it
#         ohe = [0, 0]
#         ohe[y_label] = 1
        y_label = torch.tensor(y_label)
        if self.transform: 
            img = self.transform(img)
        return (img, y_label)

In [101]:
data = Patch_Class(metadata, transform = preprocess_augmentation)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3751/3751 [00:08<00:00, 455.48it/s]


In [102]:
data.__getitem__(5)

(tensor([[[ 1.3070,  1.2385,  1.3584,  ...,  1.1187,  0.7762,  0.5878],
          [ 0.9646,  0.9988,  1.2728,  ...,  1.2385,  0.6734,  0.4508],
          [ 1.0502,  0.9817,  0.8447,  ...,  1.9064,  1.2043,  0.5707],
          ...,
          [ 0.6734,  0.5878,  0.9474,  ...,  0.9132,  0.7933,  0.7419],
          [ 0.8789,  0.9132,  0.9303,  ...,  0.9988,  0.8276,  0.7248],
          [ 1.0502,  1.1015,  0.9646,  ...,  0.7248,  0.6563,  0.6049]],
 
         [[ 0.4153,  0.3452,  0.4678,  ...,  0.2052, -0.1450, -0.3375],
          [ 0.1176,  0.1352,  0.4153,  ...,  0.6779,  0.0651, -0.2325],
          [ 0.2227,  0.1352, -0.0224,  ...,  1.4657,  0.5903, -0.0749],
          ...,
          [-0.0924, -0.1625,  0.2227,  ...,  0.2052,  0.0301, -0.0399],
          [ 0.1176,  0.1702,  0.2227,  ...,  0.2752,  0.1176, -0.0049],
          [ 0.2927,  0.3627,  0.2577,  ...,  0.0476, -0.0399, -0.0924]],
 
         [[ 1.5768,  1.4897,  1.5942,  ...,  1.1585,  0.8099,  0.6182],
          [ 1.2282,  1.2457,

# Create the Dataloader
- also subset the datasets because they're big

In [103]:
# Create the Dataloader
train_dataset, val_dataset = torch.utils.data.random_split(data, [int(len(data)*0.8), int(len(data)*0.2)+1])
print(len(train_dataset))

val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [int(len(val_dataset)*0.5), int(len(val_dataset)*0.5)+1])
print(len(val_dataset), len(test_dataset))


3000
375 376


In [104]:
batch_size = 128

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(dataset = val_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

In [105]:
len(train_loader)

24

# Load Model
- Also change the architecture slightly 

In [106]:
# torch.hub.list("pytorch/vision")

In [107]:
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

Using cache found in /dartfs-hpc/rc/home/9/f003xr9/.cache/torch/hub/pytorch_vision_main


In [108]:
#visualize the layers 
ct = 0
for child in model.children():
    print("Layer: %d" %(ct))
    print(child)
    ct += 1

Layer: 0
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Layer: 1
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Layer: 2
ReLU(inplace=True)
Layer: 3
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Layer: 4
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d

In [109]:
#we can also set the first, say, n layers to be frozen, and leave the remaining layers unfrozen, as follows 
thresh = 5
ct = 0
#here we freeze up to and including the 6th layer
for child in model.children():
    if ct <= thresh:
        for param in child.parameters():
            param.requires_grad = False
        print(child, ct)
        ct += 1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 0
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 1
ReLU(inplace=True) 2
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False) 3
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

In [110]:
#change the model architecture a bit (for vision transformer)
model.fc = nn.Sequential(nn.Linear(2048, 100), 
                         nn.ReLU(), 
                         nn.Dropout(p=.5), 
                         nn.Linear(100,2))
model

model.train()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Model Training 
- Still need to implement some standard data augmentation (i.e., rotation, flip, contrast, etc...)

In [111]:
# # Check accuracy on training to see how good our model is
# def check_accuracy(loader, model):
#     model.eval() #put model in testing
#     num_correct = 0
#     num_samples = 0
#     correct = {0:0, 1:0}
#     total = {0:0, 1:0}
#     with torch.no_grad():
#         for x, y, name in tqdm(loader):
#             #put batches on gpu 
#             x = x.to(device=device)
#             y = y.to(device=device)

#             scores = model(x)
#             _, predictions = scores.max(1)
#             for i,j in zip(predictions, y):
#                 if i.item() == j.item():
#                     correct[i.item()] +=1
#                 total[j.item()] += 1
#                 num_correct += (predictions == y).sum()
#                 num_samples += predictions.size(0)

#         print(
#               f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
#           )
#         acc = num_correct/num_samples
#         #find the accuracies for each class 
#         return acc, correct, total

#     model.train()

In [112]:
# from sklearn.metrics import roc_auc_score

# # code from: https://stackoverflow.com/questions/39685740/calculate-sklearn-roc-auc-score-for-multi-class

# def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

#     #creating a set of all the unique classes using the actual class list
#     unique_class = set([0,1,2])
#     roc_auc_dict = {}

#     for per_class in tqdm(unique_class):
#         #creating a list of all the classes except the current class 
#         other_class = [x for x in unique_class if x != per_class]

#         #marking the current class as 1 and all other classes as 0
#         new_actual_class = [0 if x in other_class else 1 for x in actual_class]
#         new_pred_class = [0 if x in other_class else 1 for x in pred_class]

#         #using the sklearn metrics method to calculate the roc_auc_score
#         roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
#         roc_auc_dict[per_class] = roc_auc

#     return roc_auc_dict

In [113]:
#hyperparams
learning_rate = 5e-4
num_epochs =25 #20 works well - it seems as tho it is a local min 

In [114]:
scaler = torch.cuda.amp.GradScaler()

from sklearn.metrics import roc_auc_score
softmax = nn.Softmax(dim=1)

Some notes
1. Might need to figure out another loss that works better with one hot encoding 
2. Also might need to figure out how to calc AUC-ROC 

In [ ]:
# Loss and optimizer
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.


criterion = nn.CrossEntropyLoss()
# criterion = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean') #experimenting with focal loss 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.1, patience=5, verbose=True)
#arrays to track the training loss and validation loss 
training_loss = []
validation_auc= []

# Train Network
for epoch in range(num_epochs):
    losses = []
    num_correct = 0
    num_samples = 0
    #train part 
    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = criterion(scores, targets)
            # print("Batch: %d. Loss: %f" %(batch_idx, loss))

        losses.append(loss.item()) # add loss
        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        # gradient descent or adam step
        scaler.step(optimizer)
        scaler.update()
    
    mean_loss = sum(losses)/len(losses)
    training_loss.append(mean_loss)
    scheduler.step(mean_loss)

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")
    
    #model in test mode 
    model.eval()
    predictions = torch.Tensor([])
    ground_truth = torch.Tensor([])
    #here, we can collect the AUC for each class
    with torch.no_grad():
        for x, y in tqdm(test_loader):
            x = x.to(device=device)
            y = y.to(device=device)
            #find the probs
            scores = softmax(model(x))
            scores = torch.argmax(scores, dim=1) #transform into indices (for multiclass)
            #move to cpu
            scores = scores.detach().cpu()
            y = y.detach().cpu()

            #concat them 
            predictions = torch.cat((predictions, scores))
            ground_truth = torch.cat((ground_truth, y))

        aucs = roc_auc_score(y, scores)
        #calc total acc here 
#         aucs = roc_auc_score_multiclass(ground_truth, predictions) #for multiclass
        print(aucs)
        validation_auc.append(aucs)
    #put the model back in train mode
    model.train()

24it [00:02, 10.43it/s]


Cost at epoch 0 is 0.17463430482894182


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 17.39it/s]


0.625


24it [00:02, 11.03it/s]


...

Cost at epoch 17 is 0.04015737026929855


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 16.55it/s]


0.7456896551724138


24it [00:02,  9.53it/s]


Cost at epoch 18 is 0.039818471879698336


# Find/Calc/and Make AUC-ROC plot

In [ ]:
model.eval()

predictions = torch.Tensor([])
ground_truth = torch.Tensor([])

with torch.no_grad():
    for x, y in tqdm(test_loader):
        x = x.to(device=device)
        y = y.to(device=device)
        #find the probs
        scores = softmax(model(x))
        scores = torch.argmax(scores, dim=1)
        #move to cpu
        scores = scores.detach().cpu()
        y = y.detach().cpu()
        
        #concat them 
        predictions = torch.cat((predictions, scores))
        ground_truth = torch.cat((ground_truth, y))
  

In [ ]:
#predict the whole test cohort AUC-ROC

# print(roc_auc_score_multiclass(ground_truth, predictions))
print(roc_auc_score(ground_truth, predictions))


In [ ]:
#from sophie's code - viz. the curve 
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

# fpr and tpr of all thresohlds
true = ground_truth
preds = predictions
fpr, tpr, threshold = metrics.roc_curve(true, preds)

#get the metrics 
roc_auc = metrics.auc(fpr, tpr)

#plot
plt.title('Test Cohort-wide AUC-ROC: class 1')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [97]:
# # fpr and tpr of all thresohlds
# true = ground_truth
# preds = probabilities[:, 0]
# fpr, tpr, threshold = metrics.roc_curve(true, preds)

# #get the metrics 
# roc_auc = metrics.auc(fpr, tpr)

# #plot
# plt.title('Test Cohort-wide AUC-ROC: class 0')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

# Save Model 

In [ ]:
# PATH = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Saved_Models/resnet50.pt"
# torch.save(model.state_dict(), PATH) # here is how we save

In [44]:
!nvidia-smi

Wed Jan  4 17:06:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   30C    P0    39W / 300W |      3MiB / 32768MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   